In [1]:
# import xml.etree.ElementTree as ET
import requests
import os
import pandas as pd


In [2]:
oc = os.getenv('OC', '')

# 자치법규

- 요청변수

    |요청변수|값|설명|
    |---|---|---|
    |OC	|string(필수)|사용자 이메일의 ID|
    |type|char(필수)|출력 형태 HTML/XML/JSON|
    |target|string : ordin(필수)|서비스 대상|
    |ID|char|자치법규ID|
    |MST|string|자치법규 일련번호|

\
- 특이사항
    - '조문여부': 'N' 이지만 '조문내용'은 있는 경우는 보통 조항 제목이다. 예시: '제6장 공개공지 등'

In [3]:
df = pd.read_csv("data/법령검색목록_자치법규_건축.csv", skiprows=1)
df.head()

,순번,자치법규MST,자치법규명,지자체명,자치법규ID,제개정구분,법령종류,공포번호,공포일자,시행일
0,1,1298118,가평군 건축물관리자의 제설 및 제빙 책임에 관한 조례,경기도 가평군,2019610,일부개정,조례,제2623호,20170731,20170731
1,2,1888393,가평군 건축물관리 조례,경기도 가평군,2205553,일부개정,조례,제3174호,20231226,20231226
2,3,2028113,가평군 건축 조례,경기도 가평군,2019611,일부개정,조례,제3296호,20250409,20250409
3,4,1967195,가평군 건축 조례 시행규칙,경기도 가평군,2019612,일부개정,규칙,제1499호,20240919,20240919
4,5,2015645,가평군 군계획 조례,경기도 가평군,2019668,일부개정,조례,제3281호,20250217,20250217


In [ ]:
id = df["자치법규ID"][1602]

url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=ordin&ID={id}&type=JSON"
response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

In [8]:
print(data.keys())
data["LawService"].keys()

dict_keys(['LawService'])


dict_keys(['자치법규기본정보', '별표', '부칙', '조문'])

In [9]:
meta = data["LawService"]["자치법규기본정보"]
meta

{'자치법규ID': '2233372',
 '자치법규명': '횡성군 수도급수 조례',
 '자치법규일련번호': '2058483',
 '지자체기관명': '강원특별자치도 횡성군',
 '시행일자': '20250714',
 '제개정정보': '일부개정',
 '자치법규종류': 'C0001',
 '공포번호': '2855',
 '전화번호': '033-340-5821',
 '담당부서명': '상하수도사업소',
 '자치법규발의종류': '',
 '공포일자': '20250714'}

In [7]:
data["LawService"]["조문"].keys()

dict_keys(['조'])

In [6]:
content = data["LawService"]["조문"]["조"]
content

[{'조문번호': ['000000', '000000'], '조제목': '', '조내용': '제1장 총칙', '조문여부': 'N'},
 {'조문번호': ['000100', '000100'],
  '조제목': '목적',
  '조내용': '제1조(목적) 이 조례는「수도법」제38조 및 「지방자치법」 제136조, 같은 법 제139조제1항 및 「지방공기업법」 제22조에 따라 횡성군의 수도요금과 급수설비에 관한 공사비의 부담구분 및 그 밖의 공급조건 등 급수의 적정관리를 위하여 필요한 사항을 규정함을 목적으로 한다.',
  '조문여부': 'Y'},
 {'조문번호': ['000200', '000200'],
  '조제목': '정의',
  '조내용': '제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음 각 호와 같다.1. "급수설비"란 수도사업자가 일반의 수요자에게 원수 또는 정수를 공급하기 위하여 설치한 배수관으로부터 분기하여 설치된 급수관(옥내급수관을 포함한다)·계량기·저수조·수도꼭지·그 밖의 급수를 위하여 필요한 기구(器具)를 말한다.2. "급수공사"란 급수설비의 신설, 개조, 수선 및 철거 등의 공사를 말한다.3. "수도사용자 등"이란 급수설비의 사용자, 소유자 및 관리인 등을 말한다.4. “중수도”란 사용한 수돗물을 생활용수·공업용수 등으로 재활용할 수 있도록 다시 처리하는 시설을 말한다.5. "공업용수"란 정수장에서 1차 침전 처리한 용수를 말한다.6. “호(戶)”란 독립적으로 생활을 영위할 수 있도록 건립된 건축단위를 말한다.7. “교육시설”이란「초·중등교육법」 제2조 각 호에 따른 학교 및 「유아교육법」 제2조제2호에 따른 유치원에서 사용하는 시설을 말한다.<개정 2019. 7. 15.>',
  '조문여부': 'Y'},
 {'조문번호': ['000300', '000300'],
  '조제목': '급수구역',
  '조내용': '제3조(급수구역)① 급수구역은 횡성군(이하 “군”이라 한다)의 관할구역 중 횡성군수(이하 "군수"라 한다)가 고시한 급수가능구역으로 한다.

In [103]:
ctx_list = []
ctx_org = []
header = ""
for ctx in content:
    if ctx["조문여부"] == "Y":
        txt = header + " " + ctx["조내용"]
        ctx_list.append(txt.strip())
        # header = ""
    else:
        header = "[" + ctx["조내용"] + "]"
    ctx_org.append(ctx["조내용"])
ctx_list

['[제1장  총    칙] 제1조(목적) 이 조례는 「건축법」,「건축법 시행령」, 「건축법 시행규칙」,「전통시장 및 상점가 육성을 위한 특별법」,「전통시장 및 상점가 육성을 위한 특별법 시행령」에서 위임한 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.<개정 2009.12.9, 2013.12.6>',
 '제2조(적용 범위) 이 조례는 거제시 행정구역의 건축물과 그 대지에 대하여 적용한다. <개정 2009.12.9>',
 '제3조(건축위원회)① 「건축법」(이하 “법”이라 한다), 「건축법 시행령」(이하 “영”이라 한다), 「건축법 시행규칙」(이하 “시행규칙”이라 한다) 및 다른 법령과 이 조례에서 건축위원회의 조사·심의를 거치도록 한 사항을 조사·심의하기 위하여 영 제5조의5제3항에 따라 위원장 및 부위원장 각 1명을 포함한 25명 이상 150명 이하의 위원으로 구성하는 거제시 건축위원회(이하 “건축위원회”라 한다)를 둔다.<개정 2013.12.6,2015.10.1., 2017.4.24.,2024.5.23., 2025.9.25.>② 건축위원회의 위원장과 부위원장은 위원 중에서 거제시장(이하 “시장”이라 한다)이 임명하거나 위촉한다. <개정 2013.12.6., 2021.9.24., 2025.9.25.>③ 건축위원회의 위원은 영 제5조의5제4항 및 제6항의 규정에 따라 시장이 임명하거나 위촉한다. 다만, 공무원을 위원으로 임명하는 경우에는 그 수를 전체위원 수의 4분의 1 이하로 구성한다.<개정 2013.12.6., 2017.4.24.,2024.5.23.>④ 위원 중 공무원이 아닌 위원의 임기는 3년으로 하되, 한 차례만 연임할 수 있다. 다만, 위원의 사임 등으로 새로 위촉된 위원의 임기는 전임 위원 임기의 남은 기간으로 한다.<개정 2013.12.6., 2021.9.24.>⑤ 건축위원회 위원의 제척ㆍ기피ㆍ회피ㆍ해임ㆍ해촉에 대해서는 영 제5조의2 및 제5조의3을 따른다. 이 경우 영 제5조의2 중 “중앙건축위원회”는 “건축위원회”로, 영 제5조의3 중

In [12]:
meta

{'자치법규ID': '2233372',
 '자치법규명': '횡성군 수도급수 조례',
 '자치법규일련번호': '2058483',
 '지자체기관명': '강원특별자치도 횡성군',
 '시행일자': '20250714',
 '제개정정보': '일부개정',
 '자치법규종류': 'C0001',
 '공포번호': '2855',
 '전화번호': '033-340-5821',
 '담당부서명': '상하수도사업소',
 '자치법규발의종류': '',
 '공포일자': '20250714'}

In [16]:
len(" ".join(ctx_list))

13352

In [75]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

model = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0.1,
    max_tokens = 1024,
    timeout=None,
    max_retries=2,
    reasoning = None,
)

system_message = """당신은 법률 문서 전문을 분석하여, 그 안에서 명시적으로 **참고해야 하는 다른 법률 문서와 해당 조항 번호**를 추출해야 합니다.

아래의 원칙을 반드시 지키세요:
1. 반드시 문서 내용에 명시적으로 등장한 법률명 또는 조항만 추출합니다.
2. 법률명이 명확히 언급되지 않은 경우, 절대 추측하지 않습니다.
3. 조항 번호나 별표만 언급된 경우, 해당 항목은 "기타" key에 넣습니다.
4. 출력은 반드시 **유효한 JSON 형식**으로 반환합니다.

다음은 약어와 실제 명칭입니다. 법률 문서 제목이나 조항명이 약어로 명시되어있는 경우, 실제 명칭으로 변환하여 반환하세요.
{abb}

출력 형식:
  "법률 문서 제목": ["관련 조항 번호", "..."],
  "기타": ["별표", "조항 번호", "..."]

주의:
- 각 리스트가 비어있다면 빈 배열([])로 반환합니다.
- 설명 문구, 해석, 자연어 문장은 절대 포함하지 마세요.
- "같은 법" 이나 "동법" 등의 표현은 실제 법률명으로 대체하세요.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{question}")
])

chain = prompt | model

# 일반 Chain 생성
chain = prompt | model | JsonOutputParser()
chain

ChatPromptTemplate(input_variables=['abb', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['abb'], input_types={}, partial_variables={}, template='당신은 법률 문서 전문을 분석하여, 그 안에서 명시적으로 **참고해야 하는 다른 법률 문서와 해당 조항 번호**를 추출해야 합니다.\n\n아래의 원칙을 반드시 지키세요:\n1. 반드시 문서 내용에 명시적으로 등장한 법률명 또는 조항만 추출합니다.\n2. 법률명이 명확히 언급되지 않은 경우, 절대 추측하지 않습니다.\n3. 조항 번호나 별표만 언급된 경우, 해당 항목은 "기타" key에 넣습니다.\n4. 출력은 반드시 **유효한 JSON 형식**으로 반환합니다 (파싱 가능한 구조).\n\n다음은 약어와 실제 명칭입니다. 법률 문서 제목이나 조항명이 약어로 명시되어있는 경우, 실제 명칭으로 변환하여 반환하세요.\n{abb}\n\n출력 형식:\n  "법률 문서 제목": ["관련 조항 번호", "..."],\n  "기타": ["별표", "조항 번호", "..."]\n\n주의:\n- 각 리스트가 비어있다면 빈 배열([])로 반환합니다.\n- 설명 문구, 해석, 자연어 문장은 절대 포함하지 마세요.\n- "같은 법" 이나 "동법" 등의 표현은 실제 법률명으로 대체하세요.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])
| ChatOllama(model='gpt-oss:120b-cl

In [242]:
model_abb = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0.1,
    max_tokens = 1024,
    timeout=None,
    max_retries=2,
    reasoning = None,
)

system_message_abb = """당신은 법률 문서 전문을 분석하여, 그 안에서 약어와 축약어의 실제 명칭을 파악 후 추출해야합니다.

아래의 원칙을 반드시 지키세요:
1. 반드시 문서 내용에 명시적으로 등장한 약어 또는 축약어만 추출합니다.
2. 약어가 명확히 언급되지 않은 경우, 절대 추측하지 않습니다.
4. 출력은 반드시 **유효한 JSON 형식**으로 반환합니다.

출력 형식:
  "약어": "실제 명칭"

주의:
- 실제 명칭은 문서 내용에 반드시 명시된 그대로 반환합니다. 실제 명칭은 여러 단어가 조합될 수 있습니다.
- 약어가 문서에 없다면 빈 JSON 객체로 반환합니다.
- 설명 문구, 해석, 자연어 문장은 절대 포함하지 마세요.
"""

prompt_abb = ChatPromptTemplate.from_messages([
    ("system", system_message_abb),
    ("user", "{question}")
])

# 일반 Chain 생성
chain_abb = prompt_abb | model_abb | JsonOutputParser()
chain_abb

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 법률 문서 전문을 분석하여, 그 안에서 약어(또는 축약어)의 실제 명칭을 정확히 추출해야 합니다.\n\n아래의 원칙을 반드시 지키세요:\n1. 반드시 문서 내용에 명시적으로 등장한 약어 또는 축약어만 추출합니다.\n2. 약어가 문서 내에서 정의되지 않은 경우, 절대 추측하거나 임의로 생성하지 않습니다.\n3. 약어가 여러 번 등장하더라도, 최초 정의된 구문(“이하 ‘~’이라 한다”)에서 실제 명칭을 추출합니다.\n4. 출력은 반드시 **유효한 JSON 형식**으로 반환해야 합니다. (파싱 가능한 구조)\n5. 실제 명칭에는 관련된 **조건문이나 수식어**(예: “법 제42조제2항에 따라 고시하는 …”)도 포함합니다.\n\n출력 형식:\n  "약어": "실제 명칭"\n\n주의:\n- 실제 명칭은 문서에 명시된 표현 그대로 반환합니다. (요약하거나 일반화하지 마세요)\n- “이하 ‘약어’라 한다” 구문에서 약어를 인식합니다.\n- 명칭 앞뒤의 **조건·수식어**(예: “~에 따라 고시하는”, “~을 의미하는”, “~으로 정하는”)가 있다면 실제 명칭에 함께 포함시킵니다.\n- 조건이 여러 문장에 흩어져 있다면, 의미가 연결되는 한도 내에서 병합하여 반환합니다.\n- 약어가 문서에 전혀 없으면 빈 JSON을 반환합니다.\n- 자연어 해석, 주석, 설명 문구는 절대 포함하지 않습니다.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input

In [278]:
model_abb = ChatOllama(
    model="gpt-oss:120b-cloud",
    temperature=0.1,
    max_tokens = 1024,
    timeout=None,
    max_retries=2,
    reasoning = None,
)

system_message_abb = """당신은 법률 문서 전문을 분석하여, 그 안에서 약어(또는 축약어)의 **원래 의미**를 추출해야 합니다.

#아래의 원칙을 반드시 지키세요:
1. 반드시 문서 내용에 **명시적으로 등장한 약어 또는 축약어만** 추출합니다.
2. 약어가 정의되지 않은 경우, 절대 추측하지 않습니다.
3. 약어가 정의된 문장은 보통 “(이하 ‘~’이라 한다)” 또는 “(이하 ‘~’라 한다)” 형태로 나타납니다.
4. 약어의 원래 의미에는 다음 요소들이 포함될 수 있습니다:
   - 관련 법령, 조항 번호, 시행령/시행규칙
   - “에 따라”, “에 의한”, “이 정하여 고시하는”, “으로 정하는”, “에서 규정한” 등의 조건문
   - 문장 속 수식어, 제약 조건 등
   이러한 조건은 **절대 생략하지 말고 그대로 포함하세요.**
5. 정의 구문에 "다만", "단서", "예외" 등이 이어지는 경우, 해당 조건도 실제 명칭에 반드시 포함합니다.
6. 출력은 반드시 **파싱 가능한 JSON 형식**으로 반환해야 합니다.

출력 형식:
  "약어": "원래 의미"

- 예시 1:
입력:
"제3조(정의) 이 조례에서 사용하는 용어의 뜻은 「건축물관리법」(이하 “법”이라 한다) 및 「건축법」에서 정하는 바에 따른다."

출력:
  "법": "건축물관리법"

- 예시 2:
입력:
"법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 “조경의무면적”이라 한다)에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준(이하 “조경기준”이라 한다)에 따라 조경의 조치를 하여야 한다. 다만, 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다. <개정 2009.12.9., 2017.11.9.>1. 연면적(대지에 둘 이상의 건축물이 있는 경우에는 연면적의 합계로 한다. 이하 이 조에서 같다)이 2천제곱미터 이상인 건축물의 조경의무면적: 대지면적의 15퍼센트 이상2. 연면적이 1천제곱미터 이상 2천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 10퍼센트 이상3. 연면적이 1천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 5퍼센트 이상② 조경기준 제4조와 제5조제1항 및 제12조에서 정하는 조경면적 산정기준이나 조경면적의 배치기준에 따라 산정한 면적을 제1항에 따른 조경의무면적으로 산정한다. 다만, 다음 각 호에서 정하는 경우에는 그 기준에 따라 산정한 면적만을 조경의무면적으로 산정하되 조경의무면적의 2분의 1을 초과할 수 없다."

출력:
  "조경의무면적": "법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준에 따라 조경의 조치를 하여야 하나, 다만 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다.",
  "조경기준": "법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준"

#출력 규칙:
- 반드시 유효한 "JSON" 형식으로만 출력하세요.
- 약어가 전혀 없을 경우에는 정확히 빈 JSON 객체만 출력하세요.
 """

prompt_abb = ChatPromptTemplate.from_messages([
    ("system", system_message_abb),
    ("user", "{question}")
])

# 일반 Chain 생성
chain_abb = prompt_abb | model_abb | JsonOutputParser()
chain_abb

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 법률 문서 전문을 분석하여, 그 안에서 약어(또는 축약어)의 **원래 의미**를 추출해야 합니다.\n\n#아래의 원칙을 반드시 지키세요:\n1. 반드시 문서 내용에 **명시적으로 등장한 약어 또는 축약어만** 추출합니다.\n2. 약어가 정의되지 않은 경우, 절대 추측하지 않습니다.\n3. 약어가 정의된 문장은 보통 “(이하 ‘~’이라 한다)” 또는 “(이하 ‘~’라 한다)” 형태로 나타납니다.\n4. 약어의 원래 의미에는 다음 요소들이 포함될 수 있습니다:\n   - 관련 법령, 조항 번호, 시행령/시행규칙\n   - “에 따라”, “에 의한”, “이 정하여 고시하는”, “으로 정하는”, “에서 규정한” 등의 조건문\n   - 문장 속 수식어, 제약 조건 등\n   이러한 조건은 **절대 생략하지 말고 그대로 포함하세요.**\n5. 정의 구문에 "다만", "단서", "예외" 등이 이어지는 경우, 해당 조건도 실제 명칭에 반드시 포함합니다.\n6. 출력은 반드시 **파싱 가능한 JSON 형식**으로 반환해야 합니다.\n\n출력 형식:\n  "약어": "원래 의미"\n\n- 예시 1:\n입력:\n"제3조(정의) 이 조례에서 사용하는 용어의 뜻은 「건축물관리법」(이하 “법”이라 한다) 및 「건축법」에서 정하는 바에 따른다."\n\n출력:\n  "법": "건축물관리법"\n\n- 예시 2:\n입력:\n"법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 “조경의무면적”이라 한다)에 법 제4

In [277]:
chain_abb.invoke({"question": "[제4장  건축물의 대지 및 도로] 제17조(대지의 조경)① 법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 “조경의무면적”이라 한다)에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준(이하 “조경기준”이라 한다)에 따라 조경의 조치를 하여야 한다. 다만, 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다. <개정 2009.12.9., 2017.11.9.>1. 연면적(대지에 둘 이상의 건축물이 있는 경우에는 연면적의 합계로 한다. 이하 이 조에서 같다)이 2천제곱미터 이상인 건축물의 조경의무면적: 대지면적의 15퍼센트 이상2. 연면적이 1천제곱미터 이상 2천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 10퍼센트 이상3. 연면적이 1천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 5퍼센트 이상② 조경기준 제4조와 제5조제1항 및 제12조에서 정하는 조경면적 산정기준이나 조경면적의 배치기준에 따라 산정한 면적을 제1항에 따른 조경의무면적으로 산정한다. 다만, 다음 각 호에서 정하는 경우에는 그 기준에 따라 산정한 면적만을 조경의무면적으로 산정하되 조경의무면적의 2분의 1을 초과할 수 없다. <개정 2009.12.9., 2013.12.6., 2015.10.1>1. 시행규칙 제26조제2항에 따라 대지조성을 위한 성토부분·절토부분 또는 되메우기를 하지 아니하는 굴착부분의 비탈면(이하 “비탈면등”이라 한다)에 환경의 보전을 위한 조치를 하여야 하는 부분 중 대지의 가장자리의 비탈면등의 평균경사도가 15도 이상인 부분은 대지면적에는 산정하나 조경의무면적의 산정에서는 제외하고, 평균경사도가 15도 미만인 비탈면등에 환경의 보전을 위한 조치를 하거나 조경의 조치를 하는 부분의 2분의 1에 해당하는 면적만을 조경의무면적에 산정한다.2. 조경기준 제3조제9호에 따른 인공지반조경부분(지상1층의 조경부분은 제외한다)은 그 조경면적의 3분의 2에 해당하는 면적만을 조경의무면적으로 산정한다.③ 삭제 <2009.12.9.>④ 영 제27조제1항제5호에 따라 대지에 염분이 함유되어 있는 경우와 건축물 용도의 특성 상 조경의 조치가 곤란하거나 불합리한 경우로서 시장이 인정하면 다음 각 호의 어느 하나에 해당하는 건축물은 조경의 조치를 하지 않을 수 있다. 다만, 제4호부터 제6호까지의 규정에 해당하는 건축물의 대지 또는 옥상광장(옥상광장을 설치하는 경우만 해당한다)에 제1항 각 호의 어느 하나에 해당하는 조경의무면적의 2분의 1 이상에 해당하는 면적에 조경기준에 따라 식재를 하거나 조경시설을 설치하여야 한다. <개정 2009.12.9.>1.「주차장법」제2조에 따른 노외주차장과 주차전용건축물2. 운동경기장(건축물이 아닌 시설물)3. 공장과 창고(지구단위계획에 따라 녹지용지가 조성되었거나 조성계획이 있는 지구단위계획구역에 건축하려는 경우만 해당한다)4. 도매시장과 소매시장5. 염분이 함유되어 있는 대지에 건축하는 건축물6. 중심상업지역과 일반상업지역에서 300제곱미터 이하의 대지에 건축하는 건축물⑤ 영 제27조제1항제10호에 따라 조경의 조치를 하지 아니할 수 있는 건축물 중 건축조례로 정하는 건축물은 같은 호 각 목의 어느 하나에 해당하는 건축물만을 말한다. <개정 2009.12.9>⑥ 조경기준에도 불구하고 건축허가의 부관(附款)에 따라 조경의 조치를 하게 할 수 있다. <개정 2009.12.9.>"})

{'조경의무면적': '법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 "조경의무면적"이라 한다)에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준에 따라 조경의 조치를 하여야 한다. 다만, 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다.',
 '조경기준': '법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준',
 '비탈면등': '시행규칙 제26조제2항에 따라 대지조성을 위한 성토부분·절토부분 또는 되메우기를 하지 아니하는 굴착부분의 비탈면(이하 "비탈면등"이라 한다)'}

In [279]:
chain_abb.invoke({"question": "[제4장  건축물의 대지 및 도로] 제17조(대지의 조경)① 법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 “조경의무면적”이라 한다)에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준(이하 “조경기준”이라 한다)에 따라 조경의 조치를 하여야 한다. 다만, 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다. <개정 2009.12.9., 2017.11.9.>1. 연면적(대지에 둘 이상의 건축물이 있는 경우에는 연면적의 합계로 한다. 이하 이 조에서 같다)이 2천제곱미터 이상인 건축물의 조경의무면적: 대지면적의 15퍼센트 이상2. 연면적이 1천제곱미터 이상 2천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 10퍼센트 이상3. 연면적이 1천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 5퍼센트 이상② 조경기준 제4조와 제5조제1항 및 제12조에서 정하는 조경면적 산정기준이나 조경면적의 배치기준에 따라 산정한 면적을 제1항에 따른 조경의무면적으로 산정한다. 다만, 다음 각 호에서 정하는 경우에는 그 기준에 따라 산정한 면적만을 조경의무면적으로 산정하되 조경의무면적의 2분의 1을 초과할 수 없다. <개정 2009.12.9., 2013.12.6., 2015.10.1>1. 시행규칙 제26조제2항에 따라 대지조성을 위한 성토부분·절토부분 또는 되메우기를 하지 아니하는 굴착부분의 비탈면(이하 “비탈면등”이라 한다)에 환경의 보전을 위한 조치를 하여야 하는 부분 중 대지의 가장자리의 비탈면등의 평균경사도가 15도 이상인 부분은 대지면적에는 산정하나 조경의무면적의 산정에서는 제외하고, 평균경사도가 15도 미만인 비탈면등에 환경의 보전을 위한 조치를 하거나 조경의 조치를 하는 부분의 2분의 1에 해당하는 면적만을 조경의무면적에 산정한다.2. 조경기준 제3조제9호에 따른 인공지반조경부분(지상1층의 조경부분은 제외한다)은 그 조경면적의 3분의 2에 해당하는 면적만을 조경의무면적으로 산정한다.③ 삭제 <2009.12.9.>④ 영 제27조제1항제5호에 따라 대지에 염분이 함유되어 있는 경우와 건축물 용도의 특성 상 조경의 조치가 곤란하거나 불합리한 경우로서 시장이 인정하면 다음 각 호의 어느 하나에 해당하는 건축물은 조경의 조치를 하지 않을 수 있다. 다만, 제4호부터 제6호까지의 규정에 해당하는 건축물의 대지 또는 옥상광장(옥상광장을 설치하는 경우만 해당한다)에 제1항 각 호의 어느 하나에 해당하는 조경의무면적의 2분의 1 이상에 해당하는 면적에 조경기준에 따라 식재를 하거나 조경시설을 설치하여야 한다. <개정 2009.12.9.>1.「주차장법」제2조에 따른 노외주차장과 주차전용건축물2. 운동경기장(건축물이 아닌 시설물)3. 공장과 창고(지구단위계획에 따라 녹지용지가 조성되었거나 조성계획이 있는 지구단위계획구역에 건축하려는 경우만 해당한다)4. 도매시장과 소매시장5. 염분이 함유되어 있는 대지에 건축하는 건축물6. 중심상업지역과 일반상업지역에서 300제곱미터 이하의 대지에 건축하는 건축물⑤ 영 제27조제1항제10호에 따라 조경의 조치를 하지 아니할 수 있는 건축물 중 건축조례로 정하는 건축물은 같은 호 각 목의 어느 하나에 해당하는 건축물만을 말한다. <개정 2009.12.9>⑥ 조경기준에도 불구하고 건축허가의 부관(附款)에 따라 조경의 조치를 하게 할 수 있다. <개정 2009.12.9.>"})

{'조경의무면적': '법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 “조경의무면적”이라 한다)에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준(이하 “조경기준”이라 한다)에 따라 조경의 조치를 하여야 한다. 다만, 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다.',
 '조경기준': '법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준',
 '비탈면등': '시행규칙 제26조제2항에 따라 대지조성을 위한 성토부분·절토부분 또는 되메우기를 하지 아니하는 굴착부분의 비탈면'}

In [259]:
chain_abb.invoke({"question": "제12조(지역건축물관리지원센터의 설치ㆍ운영)① 군수는 법 제40조제2항에 따른 지역건축물관리지원센터(이하 “건축물관리센터”라 한다)를 설치ㆍ운영할 수 있다.② 건축물관리센터는 법 제39조제3항에 따른 업무와 다음 각 호의 업무를 수행한다.1. 이 법에 따른 안전점검 의무 관리대상이 아닌 건축물의 안전관리 및 안전점검 지원에 관한 사항2. 그 밖에 군수가 건축물의 안전을 위하여 필요하다고 인정하는 경우"})

{'건축물관리센터': '법 제40조제2항에 따른 지역건축물관리지원센터'}

In [22]:
chain_abb.invoke({"question": ctx_list[2]})

{'군': '횡성군', '군수': '횡성군수'}

In [134]:
chain_abb.invoke({"question": all_chunks})

{'인허가등': '인허가·허가·승인·협의 또는 신고 등'}

In [141]:
chain_abb.invoke({"question": all_chunks})

{'시': '강릉시',
 '법': '국토의 계획 및 이용에 관한 법률',
 '시장': '강릉시장',
 '도시계획위원회': '강릉시 도시계획위원회',
 '영': '국토의 계획 및 이용에 관한 법률 시행령'}

In [51]:
meta | {"약어": abb}

{'자치법규ID': '2233372',
 '자치법규명': '횡성군 수도급수 조례',
 '자치법규일련번호': '2058483',
 '지자체기관명': '강원특별자치도 횡성군',
 '시행일자': '20250714',
 '제개정정보': '일부개정',
 '자치법규종류': 'C0001',
 '공포번호': '2855',
 '전화번호': '033-340-5821',
 '담당부서명': '상하수도사업소',
 '자치법규발의종류': '',
 '공포일자': '20250714',
 '약어': {'군': '횡성군', '군수': '횡성군수'}}

In [96]:
x

41

In [ ]:
from tqdm import tqdm
from langchain_core.documents import Document
import requests
import json

documents = []
all_ctx_list = []

for x in tqdm(range(len(df["자치법규ID"]))):
# for x in tqdm(range(1, 2)):
# for x in tqdm(range(1)):
    id = df["자치법규ID"][x]
    url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=ordin&ID={id}&type=JSON"
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()

    meta = data["LawService"]["자치법규기본정보"]
    content = data["LawService"]["조문"]["조"]

    # --- 조문별 텍스트 추출 ---
    ctx_list = []
    ctx_without_tt = []
    header = ""
    for ctx in content:
        if ctx["조문여부"] == "Y":
            txt = header + " " + ctx["조내용"]
            txt = txt.strip()
            ctx_list.append(txt)
            all_ctx_list.append(txt)
            if ctx["조내용"].find(")") != -1:
                ctx_without_tt.append(ctx["조내용"][ctx["조내용"].find(")")+1:].strip())
            else:
                ctx_without_tt.append(ctx["조내용"])
        else:
            header = "[" + ctx["조내용"] + "]"

    # --- 약어 추출 ---
    abb = {}
    meta_with_links = meta.copy()

    for i, item in enumerate(ctx_list):
        if "이하" in item:
            q = "다음 법령 내용에서 약어와 전체 명칭을 추출하세요: " + ctx_without_tt[i]
            # print(ctx_without_tt[i])
            abb_output = chain_abb.invoke({"question": q})
            # print(abb_output)

            # 그대로 유지
            filtered_abb = {
                k: v for k, v in abb_output.items()
                if (k in ["약어", ""] and v) or (k not in ["약어", ""])
            }

            # 약어 누적
            abb = abb | filtered_abb
            meta_with_links = meta_with_links | {"약어": abb}

    # --- links 추출 ---
    cnt = 1
    for i, item in enumerate(ctx_list):
        chunk = (
            "지자체기관명: " + meta["지자체기관명"]
            + ", 자치법규명: " + meta["자치법규명"]
            + "\n" + item
        )

        q = "법률 문서의 내용은 다음과 같습니다: " + ctx_without_tt[i]
        print(ctx_without_tt[i])
        links_llm = chain.invoke({"abb": abb, "question": q})

        # print(links_llm)

        # 그대로 유지
        filtered = {
            k: v for k, v in links_llm.items()
            if (k in ["기타", "법률 문서 제목"] and v) or (k not in ["기타", "법률 문서 제목"])
        }

        # --- 메타데이터 병합 ---
        meta_with_link_abb = meta_with_links.copy()
        if filtered:
            if "links" not in meta_with_link_abb:
                meta_with_link_abb["links"] = []
            meta_with_link_abb["links"].append(filtered)

        # --- Document 객체 생성 ---
        doc = Document(
            page_content=chunk,
            metadata=meta_with_link_abb,
            id=f"{meta['자치법규ID']}_chunk_{cnt}"
        )
        cnt += 1
        documents.append(doc)

    with open("documents.jsonl", "w", encoding="utf-8") as f:
        for doc in documents:
            json.dump({
                "id": doc.id,
                "page_content": doc.page_content,
                "metadata": doc.metadata
            }, f, ensure_ascii=False)
            f.write("\n")


In [95]:
len(documents)

1156

In [123]:
# import json

# with open("documents.jsonl", "w", encoding="utf-8") as f:
#     for doc in documents:
#         json.dump({
#             "id": doc.id,
#             "page_content": doc.page_content,
#             "metadata": doc.metadata
#         }, f, ensure_ascii=False)
#         f.write("\n")


In [160]:
all_ctx_list

['제1조(목적) 이 조례는 「자연재해대책법」 제27조제2항의 규정에 따라 건축물관리자의 제설·제빙에 관한 사항을 구체적으로 정함으로써 눈 또는 얼음으로 인한 주민의 불편을 최소화하고, 안전을 꾀함을 목적으로 한다.',
 '제2조(정의) 이 조례에서 사용하는 용어의 뜻은 다음과 같다. <개정 2017.7.31.>1. “도로”라 함은 「도로법」에 따른 도로, 그 밖에 일반 교통에 사용되는 모든 곳을 말한다.2. “차도”라 함은 연석선(차도와 보도를 구분하는 돌 등으로 이어진 선을 말한다), 안전표지, 그 밖에 이와 비슷한 공작물로써 그 경계를 표시하여 모든 차의 교통에 사용하도록 된 도로의 부분을 말한다.3. “보도(步道)”라 함은 연석선, 안전표지, 그 밖에 이와 비슷한 공작물로써 그 경계를 표시하여 보행자(유모차 및 신체장애인용 의자차를 포함한다)의 통행에 사용하도록 되어있는 도로의 부분을 말한다. <개정 2017.3.6.>4. “이면도로”라 함은 「도로법」에 따른 고속국도ㆍ일반국도ㆍ지방도ㆍ군도 및 농어촌도로(「농어촌도로정비법」에 따른 도로를 말한다)가 아닌 일반의 교통에 사용되는 도로로서 차도와 보도의 구분이 없는 폭 12미터 미만의 도로를 말한다.5. “보행자전용도로”라 함은 보행자만이 다닐 수 있도록 안전표지 및 그 밖에 이와 비슷한 공작물로써 표시한 도로를 말한다.6. “제설ㆍ제빙작업”이라 함은 도로상의 눈 또는 얼음을 제거하거나 녹게 하는 재료 및 모래 등을 뿌려서 보행자와 차량의 안전한 통행에 지장이 없도록 하는 것을 말한다.7. “건축물관리자”라 함은 건축물의 소유자ㆍ점유자 또는 관리자로서 그 건축물의 관리 책임이 있는 사람을 말한다.8. “시설물의 지붕”이라 함은 「자연재해대책법 시행령」제22조의8에 따른 시설물의 지붕을 말한다. <신설 2017.7.31.>',
 '제3조(건축물관리자의 제설ㆍ제빙 책임) 건축물관리자는 관리하고 있는 건축물의 대지에 접한 보도, 이면도로, 보행자전용도로, 시설물의 지붕에 대한 제설ㆍ제빙작업을 하여야 한다. <신설

## Chunk

In [280]:
from langchain_core.documents import Document
import json

def load_documents_from_jsonl(file_path):
    """
    JSONL 파일에서 Document 객체 리스트를 불러옵니다.
    
    Args:
        file_path: JSONL 파일 경로
        
    Returns:
        List[Document]: Document 객체 리스트
    """
    documents = []
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            # 빈 줄 건너뛰기
            if not line.strip():
                continue
                
            # JSON 파싱
            data = json.loads(line)
            
            # Document 객체 생성
            doc = Document(
                page_content=data["page_content"],
                metadata=data["metadata"],
                id=data["id"]
            )
            documents.append(doc)
    
    print(f"✅ 총 {len(documents)}개 Document 불러오기 완료")
    return documents

def save_documents(documents, file_path):
    """
    Document 리스트를 JSONL 파일로 저장합니다.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        for doc in documents:
            json.dump({
                "id": doc.id,
                "page_content": doc.page_content,
                "metadata": doc.metadata
            }, f, ensure_ascii=False)
            f.write("\n")
    print(f"✅ {len(documents)}개 Document를 {file_path}에 저장 완료")


In [281]:
from tqdm import tqdm
from langchain_core.documents import Document
from collections import defaultdict
import requests
import json
import os
from pathlib import Path

def extract_clause_documents(df, oc, chain_abb, chain, checkpoint_dir="checkpoints"):
    """
    조항별로 메타데이터를 추출하여 Document 리스트를 생성합니다.
    중간 저장 기능 포함.
    """
    # 체크포인트 디렉토리 생성
    Path(checkpoint_dir).mkdir(exist_ok=True)
    
    # 이미 처리된 법규 ID 확인
    processed_ids = set()
    checkpoint_file = os.path.join(checkpoint_dir, "processed_ids.txt")
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r", encoding="utf-8") as f:
            processed_ids = set(line.strip() for line in f)
        print(f"✅ 이미 처리된 법규: {len(processed_ids)}개")
    
    documents = []
    
    # 기존 documents.jsonl이 있으면 로드
    if os.path.exists("documents.jsonl"):
        print("기존 documents.jsonl 로드 중...")
        documents = load_documents_from_jsonl("documents.jsonl")
        print(f"✅ 기존 문서의 조항 리스트 {len(documents)}개 로드 완료")
    
    # 처리할 법규만 필터링
    # total_laws = len(df["자치법규ID"])
    df_law_ids = [str(id) for id in df["자치법규ID"]]
    laws_to_process = [id for id in df_law_ids if id not in processed_ids]
    
    if not laws_to_process:
        print("✅ 모든 법규가 이미 처리되었습니다.")
        return documents

    print(f"🔄 처리할 법규: {len(laws_to_process)}개 (전체 {len(df_law_ids)}개 중)")

    for id in tqdm(laws_to_process, desc="법규 처리"):
        # try:
        url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=ordin&ID={id}&type=JSON"
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        meta = data["LawService"]["자치법규기본정보"]
        content = data["LawService"]["조문"]["조"]

        # --- 조문별 텍스트 추출 ---
        ctx_list = []
        ctx_without_tt = []
        header = ""
        
        for ctx in content:
            if ctx["조문여부"] == "Y":
                txt = header + " " + ctx["조내용"]
                txt = txt.strip()
                ctx_list.append(txt)
                
                if ctx["조내용"].find(")") != -1:
                    ctx_without_tt.append(ctx["조내용"][ctx["조내용"].find(")")+1:].strip())
                else:
                    ctx_without_tt.append(ctx["조내용"])
            else:
                header = "[" + ctx["조내용"] + "]"
        # print(f"  현재 법령의 총 조항 개수: {len(ctx_list)}")
        # --- 약어 추출 ---
        abb = {}
        clause_abb_list = []  # 각 조항의 약어를 저장
        for i, item in enumerate(ctx_list):
            if "이하" in item:
                print(f"  🔤 약어 추출 중... ({i+1}/{len(ctx_list)})", end='\r')
                q = ctx_without_tt[i]
                abb_output = chain_abb.invoke({"question": q})
                
                filtered_abb = {
                    k: v for k, v in abb_output.items()
                    if (k in ["약어", ""] and v) or (k not in ["약어", ""])
                }
                abb.update(filtered_abb)
            else:
                filtered_abb = {}
            # print(f"약어: {filtered_abb}")
            clause_abb_list.append(filtered_abb)

        # --- 조항별 Document 생성 ---
        cnt = 1
        law_documents = []  # 현재 법규의 문서들
        
        for i, item in enumerate(ctx_list):
            print(f" 🔗 links 추출 중... ({i+1}/{len(ctx_list)})", end='\r')
            chunk = (
                item
            )

            q = "법률 문서의 내용은 다음과 같습니다: " + ctx_without_tt[i]
            links_llm = chain.invoke({"abb": abb, "question": q})

            filtered_links = {
                k: v for k, v in links_llm.items()
                if (k in ["기타", "법률 문서 제목"] and v) or (k not in ["기타", "법률 문서 제목"])
            }

            # --- 메타데이터 생성 ---
            doc_meta = meta.copy()
            doc_meta["약어"] = clause_abb_list[i]
            if filtered_links:
                doc_meta["links"] = [filtered_links]

            # --- Document 객체 생성 ---
            doc = Document(
                page_content=chunk,
                metadata=doc_meta,
                id=f"{meta['자치법규ID']}_clause_{cnt}"
            )
            cnt += 1
            law_documents.append(doc)
        
        # 현재 법규 문서들을 전체 리스트에 추가
        documents.extend(law_documents)
        
        # 처리 완료된 ID 기록
        processed_ids.add(id)
        with open(checkpoint_file, "a", encoding="utf-8") as f:
            f.write(f"{id}\n")
        
        # 10개 법규마다 중간 저장
        # if len(processed_ids) % 10 == 0:
        save_documents(documents, "documents.jsonl")
        # print(f"\n💾 중간 저장: {len(documents)}개 문서, {len(processed_ids)}개 법규 처리 완료")
    
        # except Exception as e:
        #     print(f"\n❌ 오류 발생 (법규 ID: {id}): {str(e)}")
        #     # 오류 로그 저장
        #     with open(os.path.join(checkpoint_dir, "error_log.txt"), "a", encoding="utf-8") as f:
        #         f.write(f"{id}: {str(e)}\n")
        #     continue

    # 최종 저장
    save_documents(documents, "documents.jsonl")
    print(f"\n✅ 최종 저장: {len(documents)}개 문서 완료")
    
    return documents

In [243]:
chain_abb.invoke({"question": "[제4장  건축물의 대지 및 도로] 제17조(대지의 조경)① 법 제42조제1항(단서 부분은 제외한다)에 따라 200제곱미터 이상인 대지에 건축을 하는 건축주는 다음 각 호의 어느 하나에 해당하는 조경면적(이하 “조경의무면적”이라 한다)에 법 제42조제2항에 따라 국토교통부장관이 고시하는 조경기준(이하 “조경기준”이라 한다)에 따라 조경의 조치를 하여야 한다. 다만, 다음 각 호의 기준보다 영 제27조제2항 각 호의 기준이 더 완화된 경우에는 그 기준에 따른다. <개정 2009.12.9., 2017.11.9.>1. 연면적(대지에 둘 이상의 건축물이 있는 경우에는 연면적의 합계로 한다. 이하 이 조에서 같다)이 2천제곱미터 이상인 건축물의 조경의무면적: 대지면적의 15퍼센트 이상2. 연면적이 1천제곱미터 이상 2천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 10퍼센트 이상3. 연면적이 1천제곱미터 미만인 건축물의 조경의무면적 : 대지면적의 5퍼센트 이상② 조경기준 제4조와 제5조제1항 및 제12조에서 정하는 조경면적 산정기준이나 조경면적의 배치기준에 따라 산정한 면적을 제1항에 따른 조경의무면적으로 산정한다. 다만, 다음 각 호에서 정하는 경우에는 그 기준에 따라 산정한 면적만을 조경의무면적으로 산정하되 조경의무면적의 2분의 1을 초과할 수 없다. <개정 2009.12.9., 2013.12.6., 2015.10.1>1. 시행규칙 제26조제2항에 따라 대지조성을 위한 성토부분·절토부분 또는 되메우기를 하지 아니하는 굴착부분의 비탈면(이하 “비탈면등”이라 한다)에 환경의 보전을 위한 조치를 하여야 하는 부분 중 대지의 가장자리의 비탈면등의 평균경사도가 15도 이상인 부분은 대지면적에는 산정하나 조경의무면적의 산정에서는 제외하고, 평균경사도가 15도 미만인 비탈면등에 환경의 보전을 위한 조치를 하거나 조경의 조치를 하는 부분의 2분의 1에 해당하는 면적만을 조경의무면적에 산정한다.2. 조경기준 제3조제9호에 따른 인공지반조경부분(지상1층의 조경부분은 제외한다)은 그 조경면적의 3분의 2에 해당하는 면적만을 조경의무면적으로 산정한다.③ 삭제 <2009.12.9.>④ 영 제27조제1항제5호에 따라 대지에 염분이 함유되어 있는 경우와 건축물 용도의 특성 상 조경의 조치가 곤란하거나 불합리한 경우로서 시장이 인정하면 다음 각 호의 어느 하나에 해당하는 건축물은 조경의 조치를 하지 않을 수 있다. 다만, 제4호부터 제6호까지의 규정에 해당하는 건축물의 대지 또는 옥상광장(옥상광장을 설치하는 경우만 해당한다)에 제1항 각 호의 어느 하나에 해당하는 조경의무면적의 2분의 1 이상에 해당하는 면적에 조경기준에 따라 식재를 하거나 조경시설을 설치하여야 한다. <개정 2009.12.9.>1.「주차장법」제2조에 따른 노외주차장과 주차전용건축물2. 운동경기장(건축물이 아닌 시설물)3. 공장과 창고(지구단위계획에 따라 녹지용지가 조성되었거나 조성계획이 있는 지구단위계획구역에 건축하려는 경우만 해당한다)4. 도매시장과 소매시장5. 염분이 함유되어 있는 대지에 건축하는 건축물6. 중심상업지역과 일반상업지역에서 300제곱미터 이하의 대지에 건축하는 건축물⑤ 영 제27조제1항제10호에 따라 조경의 조치를 하지 아니할 수 있는 건축물 중 건축조례로 정하는 건축물은 같은 호 각 목의 어느 하나에 해당하는 건축물만을 말한다. <개정 2009.12.9>⑥ 조경기준에도 불구하고 건축허가의 부관(附款)에 따라 조경의 조치를 하게 할 수 있다. <개정 2009.12.9.>"})

{'조경의무면적': '조경면적', '조경기준': '조경기준', '비탈면등': '비탈면'}

In [282]:
def merge_documents_by_chapter(documents, min_len=300, max_len=600, 
                               checkpoint_dir="checkpoints", batch_size=1):
    """
    조항별 Document를 장 단위로 병합하여 적절한 크기의 청크를 생성합니다.
    중간 저장 기능 포함.
    """
    # 체크포인트 디렉토리 생성
    Path(checkpoint_dir).mkdir(exist_ok=True)
    
    # 이미 병합된 문서가 있으면 로드
    merged_docs = []
    if os.path.exists("documents_merged.jsonl"):
        print("기존 documents_merged.jsonl 로드 중...")
        merged_docs = load_documents_from_jsonl("documents_merged.jsonl")
        print(f"✅ 기존 병합 문서 청크 리스트{len(merged_docs)}개 로드 완료")
    
    # 이미 처리된 법규 ID 확인
    processed_law_ids = set()
    merge_checkpoint_file = os.path.join(checkpoint_dir, "merged_law_ids.txt")
    if os.path.exists(merge_checkpoint_file):
        with open(merge_checkpoint_file, "r", encoding="utf-8") as f:
            processed_law_ids = set(line.strip() for line in f if line.strip())
        print(f"✅ 이미 병합된 법규: {len(processed_law_ids)}개")
    
    # 1️⃣ Document들을 자치법규ID 기준으로 그룹화
    docs_by_law = defaultdict(list)
    for doc in documents:
        law_id = str(doc.metadata.get("자치법규ID"))
        if law_id not in processed_law_ids:
            docs_by_law[law_id].append(doc)
    
    if not docs_by_law:
        print("✅ 모든 법규가 이미 병합되었습니다.")
        return merged_docs

    # 2️⃣ 각 법령 문서별로 처리
    total_laws = len(docs_by_law)
    processed_count = 0
    
    print(f"🔄 병합할 법규: {total_laws}개")
    
    for law_id, doc_list in tqdm(docs_by_law.items(), desc="법령별 병합"):
        try:
            # ID를 숫자 기준으로 정렬 (2019610_clause_1, 2019610_clause_2, ...)
            def extract_clause_number(doc):
                # ID 형식: "2019610_clause_1"
                parts = doc.id.split("_")
                if len(parts) >= 3:
                    try:
                        return int(parts[2])  # clause 번호
                    except:
                        return 0
                return 0
            
            doc_list = sorted(doc_list, key=extract_clause_number)
            base_meta = {k: v for k, v in doc_list[0].metadata.items() 
                         if k not in ["links", "약어"]}

            # 3️⃣ 장별로 그룹화
            chapters = []
            current_chapter = []
            current_header = ""
            
            for doc in doc_list:
                content = doc.page_content
                
                # 장 헤더 추출: [제1장 총칙] 형태
                header = ""
                if content.startswith("[") and "]" in content:
                    # 첫 번째 ']'까지가 장 헤더
                    end_idx = content.find("]")
                    potential_header = content[:end_idx+1]
                    # "제1장", "제2장" 같은 패턴이 있으면 장 헤더로 인식
                    if "장" in potential_header or "편" in potential_header:
                        header = potential_header
                
                # 헤더가 바뀌면 새 장 시작
                if header and header != current_header:
                    if current_chapter:
                        chapters.append({
                            "header": current_header,
                            "docs": current_chapter
                        })
                    current_header = header
                    current_chapter = [doc]
                else:
                    current_chapter.append(doc)
            
            # 마지막 장 추가
            if current_chapter:
                chapters.append({
                    "header": current_header,
                    "docs": current_chapter
                })

            # 4️⃣ 각 장별로 청크 생성
            for chapter_idx, chapter in enumerate(chapters, start=1):
                지자체기관명 = base_meta.get("지자체기관명", "")
                자치법규명 = base_meta.get("자치법규명", "")
                
                # 장 헤더가 있으면 포함
                if chapter["header"]:
                    base_header = f"[{지자체기관명} | {자치법규명}]\n{chapter['header']}"
                else:
                    base_header = f"[{지자체기관명} | {자치법규명}]"
                
                buffer = base_header
                merged_links_dict = defaultdict(set)
                merged_abb = {}
                chunk_id = 1
                
                for doc in chapter["docs"]:
                    content = doc.page_content
                    
                    # 장 헤더 제거 (조문 내용만 추출)
                    text = content
                    if content.startswith("[") and "]" in content:
                        # [제1장 총칙] 제1조... 형태에서 제1조... 부분만 추출
                        end_idx = content.find("]")
                        potential_header = content[:end_idx+1]
                        # 장 헤더인 경우에만 제거
                        if "장" in potential_header or "편" in potential_header:
                            text = content[end_idx+1:].strip()
                    
                    # 링크 병합
                    links_list = doc.metadata.get("links", [])
                    for links in links_list:
                        for key, values in links.items():
                            if isinstance(values, list):
                                merged_links_dict[key].update(values)
                            else:
                                merged_links_dict[key].add(values)
                    
                    # 약어 병합
                    abb = doc.metadata.get("약어", {})
                    merged_abb.update(abb)
                    
                    # 버퍼에 추가할지 결정
                    potential_length = len(buffer) + len(text) + 1
                    
                    if potential_length <= max_len:
                        buffer += f"\n{text}"
                    else:
                        # 현재 버퍼가 min_len 미만이면 강제로 추가
                        if len(buffer) < min_len:
                            buffer += f"\n{text}"
                        else:
                            # 현재 버퍼로 Document 생성
                            merged_doc = create_merged_document(
                                law_id=law_id,
                                chapter_idx=chapter_idx,
                                chunk_id=chunk_id,
                                content=buffer.strip(),
                                base_meta=base_meta,
                                links_dict=merged_links_dict,
                                abb=merged_abb
                            )
                            merged_docs.append(merged_doc)
                            
                            # 새 버퍼 시작
                            chunk_id += 1
                            buffer = base_header + f"\n{text}"
                            merged_links_dict = defaultdict(set)
                            merged_abb = {}
                            
                            # 현재 조항의 링크와 약어 추가
                            for links in links_list:
                                for key, values in links.items():
                                    if isinstance(values, list):
                                        merged_links_dict[key].update(values)
                                    else:
                                        merged_links_dict[key].add(values)
                            merged_abb.update(abb)
                
                # 마지막 버퍼 처리
                if len(buffer.strip()) > len(base_header.strip()):
                    merged_doc = create_merged_document(
                        law_id=law_id,
                        chapter_idx=chapter_idx,
                        chunk_id=chunk_id,
                        content=buffer.strip(),
                        base_meta=base_meta,
                        links_dict=merged_links_dict,
                        abb=merged_abb
                    )
                    merged_docs.append(merged_doc)
            
            # 처리 완료된 법규 ID 기록
            processed_law_ids.add(str(law_id))
            with open(merge_checkpoint_file, "a", encoding="utf-8") as f:
                f.write(f"{str(law_id)}\n")
            
            processed_count += 1
            
            # batch_size개 법규마다 중간 저장
            if processed_count % batch_size == 0:
                save_documents(merged_docs, "documents_merged.jsonl")
                # print(f"\n💾 중간 저장: {len(merged_docs)}개 병합 문서, {processed_count}/{total_laws}개 법규 처리 완료")
        
        except Exception as e:
            print(f"\n❌ 병합 오류 (법규 ID: {law_id}): {str(e)}")
            import traceback
            print(traceback.format_exc())
            # 오류 로그 저장
            with open(os.path.join(checkpoint_dir, "merge_error_log.txt"), "a", encoding="utf-8") as f:
                f.write(f"{law_id}: {str(e)}\n{traceback.format_exc()}\n")
            continue

    # 최종 저장
    save_documents(merged_docs, "documents_merged.jsonl")
    print(f"\n✅ 최종 병합 저장: {len(merged_docs)}개 문서 완료")
    
    return merged_docs

In [283]:
def create_merged_document(law_id, chapter_idx, chunk_id, content, base_meta, links_dict, abb):
    """
    병합된 Document 객체를 생성합니다.
    """
    metadata = base_meta.copy()
    
    # links 정리 (set을 list로 변환)
    if links_dict:
        metadata["links"] = [{k: sorted(list(v)) for k, v in links_dict.items()}]
    
    # 약어 추가
    if abb:
        metadata["약어"] = abb
    
    doc = Document(
        page_content=content,
        metadata=metadata,
        id=f"{law_id}_chapter{chapter_idx}_chunk{chunk_id}"
    )
    
    return doc

In [284]:
def save_documents(documents, file_path):
    """
    Document 리스트를 JSONL 파일로 저장합니다.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        for doc in documents:
            json.dump({
                "id": doc.id,
                "page_content": doc.page_content,
                "metadata": doc.metadata
            }, f, ensure_ascii=False)
            f.write("\n")

In [285]:
def load_documents_from_jsonl(file_path):
    """
    JSONL 파일에서 Document 객체 리스트를 불러옵니다.
    """
    documents = []
    
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
                
            data = json.loads(line)
            doc = Document(
                page_content=data["page_content"],
                metadata=data["metadata"],
                id=data["id"]
            )
            documents.append(doc)
    
    return documents

In [ ]:
# --- 메인 실행 ---
if __name__ == "__main__":
    # 1단계: 조항별 메타데이터 추출
    print("=" * 50)
    print("1단계: 조항별 메타데이터 추출 중...")
    print("=" * 50)
    clause_documents = extract_clause_documents(df[:200], oc, chain_abb, chain)
    print(f"\n✅ 조항별 문서 {len(clause_documents)}개 완료")

    # # 2단계: 장별 병합
    # print("\n" + "=" * 50)
    # print("2단계: 장별 병합 중...")
    # print("=" * 50)
    # merged_documents = merge_documents_by_chapter(
    #     clause_documents, 
    #     min_len=300, 
    #     max_len=600,
    #     batch_size=100  # 100개 법규마다 저장
    # )
    # print(f"\n✅ 병합 문서 {len(merged_documents)}개 완료")

    # print("\n" + "=" * 50)
    # print("🎉 모든 작업 완료!")
    # print(f"조항별 문서: {len(clause_documents)}개 → documents.jsonl")
    # print(f"병합 문서: {len(merged_documents)}개 → documents_merged.jsonl")
    # print("=" * 50)

In [12]:
# Qdrant 클라이언트 설치
# pip install qdrant-client

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from qdrant_client.models import Filter, FieldCondition, MatchValue

QDRANT_URL = os.getenv('QDRANT_URL', '')
QDRANT_api_key = os.getenv('QDRANT_api_key', '')

# Qdrant 클라이언트 초기화 (로컬 또는 클라우드)
# client = QdrantClient(host="localhost", port=6333)  # 로컬 사용 시
# 또는
# client = QdrantClient(url="https://your-cluster.qdrant.io", api_key="your-api-key")  # 클라우드 사용 시


In [14]:
qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_api_key,
)

print(qdrant_client.get_collections())

collections=[]


In [117]:
from langchain_community.embeddings import ClovaXEmbeddings

clova_embedder = ClovaXEmbeddings(
                model="bge-m3",
                api_key=os.getenv("CLOVASTUDIO_API_KEY")
                )

In [120]:
clova_embedder.

ClovaXEmbeddings(client=<httpx.Client object at 0x128bd4370>, async_client=<httpx.AsyncClient object at 0x128bd4ca0>, ncp_clovastudio_api_key=SecretStr('**********'), ncp_apigw_api_key=None, base_url='https://clovastudio.stream.ntruss.com', app_id=None, service_app=False, model_name='bge-m3', timeout=60)

In [ ]:
collection_name = "local_regulations"

# 컬렉션 생성 (벡터 차원은 임베딩 모델에 따라 조정)
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=clova_embedder, distance=Distance.COSINE),  # OpenAI ada-002 기준
)


## 목록 조회

- target: 서비스 대상
    - `eflaw`: 현행법령(시행일)

In [ ]:
# api로 현행법령(시행일) 목록 조회하면 누락되는게 많음.
url = f"http://www.law.go.kr/DRF/lawSearch.do?OC={oc}&target=law&type=JSON&query=건축"

response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

In [86]:
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID=010787&type=JSON"

response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

{'법령': {'개정문': {'개정문내용': [['⊙대통령령 제35811호(2025.10.1)', '정부조직 개편 반영을 위한 306개 법령의 일부개정에 관한 대통령령', '제1조부터 제169조까지 생략', '제170조(「건축기본법 시행령」의 개정) 건축기본법 시행령 일부를 다음과 같이 개정한다.', '  제5조제10호 및 제12호를 각각 다음과 같이 한다.', '    10. 산업통상부장관', '    12. 기후에너지환경부장관', '  제18조제1항 중 "산업통상자원부장관"을 "산업통상부장관"으로 한다.', '  제23조제3항제1호사목을 다음과 같이 한다.', '      사. 산업통상부', '제171조부터 제306조까지 생략', '          부칙', '이 영은 공포한 날부터 시행한다. 다만, 이 영에 따라 개정되는 대통령령 중 이 영 시행 전에 공포되었으나 시행일이 도래하지 아니한 대통령령을 개정한 부분은 각각 해당 대통령령의 시행일부터 시행한다.']]}, '법령키': '0107872025100135811', '기본정보': {'법령명_한글': '건축기본법 시행령', '공포번호': '35811', '전화번호': '044-201-3766', '언어': '한글', '제개정구분': '타법개정', '법령ID': '010787', '공동부령정보': '', '소관부처': {'content': '국토교통부', '소관부처코드': '1613000'}, '공포법령여부': 'N', '법종구분': {'content': '대통령령', '법종구분코드': 'A0007'}, '시행일자': '20251001', '연락부서': {'부서단위': {'부서연락처': '044-201-3766', '부서키': '391623', '부서명': '건축정책과', '소관부처명': '국토교통부', '소관부처코드': '1613000'}}, '법령명_한자': '건축기본법 시행령', '법령명약칭': '', '공포일자': '20251001', '편장절관': '34030000'}, '부칙': {'부칙단위': 

### HTML

In [ ]:
from bs4 import BeautifulSoup

url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=HTML"

# 1️⃣ URL 요청
response = requests.get(url)
response.raise_for_status()
html = response.text  # HTML 본문

# 2️⃣ BeautifulSoup으로 파싱
soup = BeautifulSoup(html, "html.parser")

In [38]:
# 3️⃣ iframe 추출
iframe = soup.find("iframe")
if iframe:
    iframe_url = iframe.get("src")
    print("실제 법령 본문 URL:", iframe_url)
else:
    print("iframe을 찾을 수 없습니다.")

실제 법령 본문 URL: http://www.law.go.kr/LSW/lsInfoP.do?lsiSeq=276923&chrClsCd=010202&urlMode=lsInfoP&efYd=&ancYnChk=0&mobile=&gubun=api


In [39]:
iframe_url

'http://www.law.go.kr/LSW/lsInfoP.do?lsiSeq=276923&chrClsCd=010202&urlMode=lsInfoP&efYd=&ancYnChk=0&mobile=&gubun=api'

In [40]:
inner_html = requests.get(iframe_url).text
inner_soup = BeautifulSoup(inner_html, "html.parser")

# 실제 법령 본문 조문 추출 (예시)
for article in inner_soup.select(".pgroup"):
    print(article.get_text(strip=True))

In [58]:
res = requests.post("https://www.law.go.kr/LSW/lsInfoP.do?lsiSeq=276923#0000")
soup = BeautifulSoup(res.text, "html.parser")
law_texts = [x.get_text(strip=True) for x in soup.select(".pgroup, .pgroup2, .pgroup3")]
print(law_texts[:5])

[]


In [53]:
'<div id="lsBdy">' in res.text

False

In [23]:
url = f"http://www.law.go.kr/DRF/lawService.do?OC={oc}&target=eflaw&ID={id}&type=JSON"

response = requests.get(url)
response.raise_for_status()

data = response.json()
print(data)

{'법령': {'개정문': {'개정문내용': [['⊙법률 제21065호(2025.10.1)', '정부조직법 일부개정법률', '[본문 생략]', '        부칙', '제1조(시행일) 이 법은 공포한 날부터 시행한다. 다만, 부칙 제7조에 따라 개정되는 법률 중 이 법 시행 전에 공포되었으나 시행일이 도래하지 아니한 법률을 개정한 부분은 각각 해당 법률의 시행일부터ㆍㆍㆍ<생략>ㆍㆍㆍ 시행한다.', '  1. 및 2. 생략', '제2조부터 제6조까지 생략', '제7조(다른 법률의 개정) ①부터 <436>까지 생략', '  <437> 건축기본법 일부를 다음과 같이 개정한다.', '  제20조 각 호 외의 부분 및 제21조제1항ㆍ제2항 중 "산업통상자원부장관"을 각각 "산업통상부장관"으로 한다.', '  <438>부터 <626>까지 생략', '제8조 생략']]}, '법령키': '0105942025100121065', '기본정보': {'법령명_한글': '건축기본법', '공포번호': '21065', '전화번호': '044-201-3766', '언어': '한글', '제개정구분': '타법개정', '법령ID': '010594', '공동부령정보': '', '소관부처': {'content': '국토교통부', '소관부처코드': '1613000'}, '공포법령여부': 'N', '법종구분': {'content': '법률', '법종구분코드': 'A0002'}, '시행일자': '20251001', '연락부서': {'부서단위': {'부서연락처': '044-201-3766', '부서키': '388805', '부서명': '건축정책과', '소관부처명': '국토교통부', '소관부처코드': '1613000'}}, '법령명_한자': '건축기본법', '법령명약칭': '', '공포일자': '20251001', '편장절관': '34030000'}, '부칙': {'부칙단위': [{'부칙키': '2007122108783', '부칙공포일자': '20071221', '부칙내용': [['부칙 <제8783호,20

In [26]:
data.keys()

dict_keys(['법령'])

In [27]:
data['법령'].keys()

dict_keys(['개정문', '법령키', '기본정보', '부칙', '조문', '제개정이유'])

In [30]:
data

False